In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'    
}

In [5]:
movies = []
for i in range(0,250,25):
    url = f'https://movie.douban.com/top250?start={i}'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content,'html.parser')
    movies += soup.find_all('div',class_='info')

In [6]:
for i,movie in enumerate(movies):
    title = movie.find('span',class_= 'title').text.strip()
    rating_num = movie.find('span',class_='rating_num').text.strip()
    link = movie.find('a')['href']
    print(f'{i + 1}. {title} - {rating_num} - {link}')    

1. 肖申克的救赎 - 9.7 - https://movie.douban.com/subject/1292052/
2. 霸王别姬 - 9.6 - https://movie.douban.com/subject/1291546/
3. 阿甘正传 - 9.5 - https://movie.douban.com/subject/1292720/
4. 泰坦尼克号 - 9.5 - https://movie.douban.com/subject/1292722/
5. 千与千寻 - 9.4 - https://movie.douban.com/subject/1291561/
6. 这个杀手不太冷 - 9.4 - https://movie.douban.com/subject/1295644/
7. 美丽人生 - 9.5 - https://movie.douban.com/subject/1292063/
8. 星际穿越 - 9.4 - https://movie.douban.com/subject/1889243/
9. 盗梦空间 - 9.4 - https://movie.douban.com/subject/3541415/
10. 楚门的世界 - 9.4 - https://movie.douban.com/subject/1292064/
11. 辛德勒的名单 - 9.5 - https://movie.douban.com/subject/1295124/
12. 忠犬八公的故事 - 9.4 - https://movie.douban.com/subject/3011091/
13. 海上钢琴师 - 9.3 - https://movie.douban.com/subject/1292001/
14. 三傻大闹宝莱坞 - 9.2 - https://movie.douban.com/subject/3793023/
15. 放牛班的春天 - 9.3 - https://movie.douban.com/subject/1291549/
16. 机器人总动员 - 9.3 - https://movie.douban.com/subject/2131459/
17. 疯狂动物城 - 9.2 - https://movie.douban.com/su

In [7]:
import pandas as pd

movies_list = []

for i, movie in enumerate(movies):
    url = movie.find('a')['href'] 
    response = requests.get(url, headers=headers)  
    soup = BeautifulSoup(response.content, 'html.parser')
    
    info = soup.find('div', id='info')  
    
    directors = [d.text.strip() for d in info.find_all('a', rel='v:directedBy')]
    writers = [w.text.strip() for w in info.find_all('span', class_='attrs', text='编剧')]
    actors = [a.text.strip() for a in soup.find_all('a', rel='v:starring')]
    genres = [g.text.strip() for g in info.find_all('span', property='v:genre')]
    release_date = info.find('span', property='v:initialReleaseDate').text.strip()
    runtime = info.find('span', property='v:runtime').text.strip()
    rating_count = soup.find('span', property='v:votes').text.strip()
    summary = soup.find('span', property='v:summary').text.strip()

    movie_data = {
        'Title': movie.find('a').text.strip(),
        'Directors': ", ".join(directors),
        'Writers': ", ".join(writers),
        'Actors': ", ".join(actors),
        'Genres': ", ".join(genres),
        'Release Date': release_date,
        'Runtime': runtime,
        'Rating Count': rating_count,
        'Summary': summary
    }
    movies_list.append(movie_data)

df = pd.DataFrame(movies_list)

df.to_csv('movies.csv', index=False, encoding='utf-8')

C:\Users\Bingjie Jiang\AppData\Local\Temp\ipykernel_18232\1348376593.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  writers = [w.text.strip() for w in info.find_all('span', class_='attrs', text='编剧')]


In [8]:
import pandas as pd
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Function to translate text
def translate_text(text, src='zh-CN', dest='en'):
    try:
        translation = translator.translate(text, src=src, dest=dest)
        return translation.text
    except Exception as e:
        return text  # Return original text if translation fails

# Load the CSV file
file_path = 'douban movies.csv'
df = pd.read_csv(file_path, encoding='gb18030')

# Specify the columns to translate
columns_to_translate = ['Title', 'Directors', 'Actors', 'Genres', 'Summary']

# Translate the content of each relevant column
for column in columns_to_translate:
    df[column] = df[column].apply(lambda x: translate_text(x) if pd.notnull(x) else x)

# Save the translated dataframe to a new CSV file
translated_file_path = 'translated_douban_movies.csv'
df.to_csv(translated_file_path, index=False, encoding='utf-8')

print(f'Translation complete. Translated file saved to {translated_file_path}')

Translation complete. Translated file saved to translated_douban_movies.csv
